In [1]:
import os
import pandas as pd
import numpy as np
import re
import sqlite3
import string


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
pip install git+https://github.com/huggingface/transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-jgr4qlo8
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-jgr4qlo8
  Resolved https://github.com/huggingface/transformers to commit 65cf33e7e53cd46313f3655f274b3f6ca0fd679d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 67 kB 58.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.5 MB 1.2 MB/s eta 0:00:01
  Created wheel for transformers: filename=transformers-4.18.0.dev0-py3-none-any.whl size=3836149 sha256=2b22f0121624af0c26225f1a5fea58562d06baef005b75ebad89eeb699a26e0f
  Stored in directory: /tmp/pip-ephem-wheel-cache-eu8n76tv/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers
Note: you may need to restart the kernel to

In [13]:
# print(df_tweet['text'])

# con = sqlite3.connect('/content/drive/MyDrive/labelled_data.db')
con = sqlite3.connect('labelled_data.db')
cur = con.cursor()
res = cur.execute("SELECT * FROM labelled_data;").fetchall()
temp_res = cur.execute("SELECT * FROM labelled_data;").fetchall()

In [14]:
# Problem with the utf-8 characters: original input file recognize these slashes as a string, not a special character
# whereas if you initialize a var = str('/xe9'), it will still recognize this slash as a special character for utf-8
# so this will be different from the utf-8 code we extract from database.

# asdf = cur.execute("SELECT text FROM Tweets_Data_Unique where keyword = 'flower' AND status_id = 'b''1262997263690813441''';").fetchall()
# asdf[0] = list(asdf[0])
# asdf[0][0] = asdf[0][0][2:]
# aaaa = str(asdf[0][0])
# aaaa = str(aaaa[66:90])
# llll = "\xed\x8c\x90\xeb\xa7\xa4"
# print(aaaa == llll)
# print(aaaa.encode('utf-8').decode('utf-8', 'ignore'))
# print(llll.encode('utf-8').decode('utf-8', 'ignore'))

# My solution is just to remove them all, since there is no reason to recognize emojis in NLP


def removeUrl(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)
    
def removeEmoji(text):
#     emojis = re.findall(r'\\x\S+',text)
#     return emojis
#     emojis = re.compile(r'\S+(\\n+)|\\x\S+(#+)|\\x\S+')
    emojis = re.compile(r'\S?\\x(\\n+)|\\x\S+(#+)|\\x\S+')
    return emojis.sub(r'',text)

def removePunc(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

def removeSpecialChar(text):
    nxtln = re.compile(r'\S?\\n')
    ans = nxtln.sub(r' ',text)
    rt = re.compile(r'RT\S?')
    newAns = rt.sub(r'', ans)
    at = re.compile(r'@\S+')
    return at.sub(r'', newAns)
    
# -----------------------------Test Section-----------------------------
# Data preprocessing
# asdf = cur.execute("SELECT text FROM labelled_data where id = '374';").fetchall()
# asdf[0] = list(asdf[0])
# asdf[0][2] = asdf[0][0][2:]
# print(asdf[0][0])
# asdf[0][0] = removeUrl(asdf[0][0])
# print("after removing url: " + asdf[0][0])
# asdf[0][0] = removeEmoji(asdf[0][0])
# print("after removing emoji: " + asdf[0][0])
# asdf[0][0] = removeSpecialChar(asdf[0][0])
# print("after removing specialchar: " + asdf[0][0])

# asdf[0][0] = ' '.join(asdf[0][0].split())
# print(asdf[0][0])





# -----------------------------Production Section-----------------------------
# https://realpython.com/python-encodings-guide/
# this website tells how to decode these utf-8 symbols
# b" means it is in byte format, so it will need to be in bytes, but we only have strings
# print(b"\xf0\x9f\x8f\x88\xf0\x9f\x92\xaa\xf0\x9f\x8f\xbe".decode("utf-8"))

# limit the amount of data to 320
# data_320 = []
# for i in range(320):
#     things = list(res[i])
#     things[2] = things[2][2:]
#     things[2] = removeUrl(things[2])
#     things[2] = removeEmoji(things[2])
#     things[2] = removeSpecialChar(things[2])
#     things[2] = ' '.join(things[2].split())
#     things = things[2]
#     if len(things) == 0:
#       things = "None"
#     data_320.append(things)



# remove all unnecessary stuff + Lemmatization
for i in range(len(res)):
    res[i] = list(res[i])
    res[i][2] = res[i][2][2:]
    res[i][2] = removeUrl(res[i][2])
    res[i][2] = removeEmoji(res[i][2])
    res[i][2] = removeSpecialChar(res[i][2])
    res[i][2] = ' '.join(res[i][2].split())
    res[i] = res[i][2]
    if len(res[i]) == 0:
      res[i] = "None"


con.commit()
con.close()


In [9]:
pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 121 kB 36.3 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 76.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 61.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")

# model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli").to("cuda:0")

bart_large_mnli_pipeline = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=0)

candidate_labels = ['phenology', 'sports', 'education', 'comments', 'electronics', 'music', 'movies', 'art', 'philosophy']


Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [25]:
# print(bart_large_mnli_pipeline(res[0], candidate_labels, batch_size=8)['labels'])
print(len(res))
print(len(temp_res))
# print(len(data_320))

124000
124000


In [16]:
from tqdm.notebook import tqdm
import time




# 40s for 320 data with 320 batch_size
# 27s for 320 data with 16 batch_size??
# 29~30s for 320 data with 32 batch_size
# 31s for 320 data with 64 batch_size
# 28s for 320 data with 8 batch_size

# 55s for 640 data with 16 batch_size √
start = time.time()
# ans_after_test = bart_large_mnli_pipeline(data_320, candidate_labels, batch_size=16)
ans_after_bart = bart_large_mnli_pipeline(res, candidate_labels, batch_size=16)
stop = time.time()
print("The time of the run:", stop - start)

The time of the run: 2433.2728641033173


In [18]:
print(res[0])

The first bloom of my newest addition! It took its sweet time because it was less excited about blooming than I was. http


In [19]:
print(temp_res[0])

(1576, "b'1263178736729567232'", "b'RT @nikkigrimes9: The first bloom of my newest addition! It took its sweet time because it was less excited about blooming than I was. http\\xe2\\x80\\xa6'", 'bloom', 1, 1)


In [31]:
cnt_res = 0
categorized_res =[]
all_answers = []

for pipeln in ans_after_bart:
  temp = [1, temp_res[cnt_res][2], pipeln['labels'],  pipeln['scores']]
  if pipeln['labels'][0] == 'phenology' and pipeln['scores'][0] >= 0.3:
    categorized_res.append(1)
  else:
    temp[0] = 0
    categorized_res.append(0)
  all_answers.append(temp)
  cnt_res+=1


In [32]:
# print(len(categorized_res) == len(res) == len(all_answers) )
print(len(categorized_res))
print(len(all_answers))

answer_list = [temp_res[i][4] for i in range(len(temp_res))]
# answer_list = [temp_res[i][4] for i in range(len(data_300))]
print(len(answer_list))

total_ones = answer_list.count(1);
print(total_ones)
print(len(all_answers))

124000
124000
124000
1242
124000


In [33]:
true_pos = 0
true_neg = 0
false_pos = 0
false_neg = 0


print("False Negatives for Bart Zero-Shot learning: ")
fn_file = open("false_negative_bart.txt", "w+")
for i in range(len(categorized_res)):
  if categorized_res[i] == answer_list[i]:
    # predict 1, actual 1
    if categorized_res[i] == 1:
      true_pos+=1
    # predict 0, actual 0
    else:
      true_neg+=1
  else:
    # predict 1, actual 0
    if categorized_res[i] == 1:
      false_pos +=1
    # predict 0, actual 1
    else:
      false_neg +=1
      original_tweet = all_answers[i][1]
      result_label_order = all_answers[i][2]
      result_label_weight = all_answers[i][3]
      fn_file.write("Original Tweet: {}".format(str(original_tweet)))
      fn_file.write("Result label order: {}".format(str(result_label_order)))
      fn_file.write("Result label weights: {}".format(str(result_label_weight)))
      fn_file.write('\n')



False Negatives for Bart Zero-Shot learning: 
With Bart Zero Shot Learning, number of tweets: 300
true_pos:  754
false_pos:  17143
false_neg:  488
true_neg:  105615
total # of positive labels:  1242
correctness:  0.8578145161290323


In [35]:
print("==================")
print("With Bart Zero Shot Learning, number of tweets: 300")
print("true_pos: ", true_pos)
print("false_pos: ", false_pos)
print("false_neg: ", false_neg)
print("true_neg: ", true_neg)
print("total # of positive labels: ", total_ones)
print("total # of data: ", len(categorized_res))
print("accuracy: ", (true_pos+true_neg)/len(categorized_res))
print("precision: ", true_pos/(true_pos+false_pos))
print("recall: ", true_pos/(true_pos+false_neg))

With Bart Zero Shot Learning, number of tweets: 300
true_pos:  754
false_pos:  17143
false_neg:  488
true_neg:  105615
total # of positive labels:  1242
total # of data:  124000
accuracy:  0.8578145161290323
precision:  0.04212996591607532
recall:  0.607085346215781
